# 使用__slots__
## 给实例绑定属性和方法的一些方式

In [7]:
class Student(object):
    pass

s=Student()

#动态给实例绑定一个属性
s.name='Michael'

print(s.name)

Michael


In [8]:
#但这在另一个实例上则不生效
s1=Student()
print(s1.name)

AttributeError: 'Student' object has no attribute 'name'

In [9]:
#可以通过给类绑定属性实现效果
Student.name='Michael'
print(s1.name)

Michael


In [10]:
#同样可以实现方法的动态绑定
def set_score(self,score):
    self.score=score

#绑定方法
from types import MethodType    #用于将实例与方法绑定的函数
s.set_score=MethodType(set_score,s)

s.set_score(25)
print(s.score)

25


In [11]:
#同样的需要对类绑定才能作用在所有实例
Student.set_score=set_score

s1.set_score(10)
print(s1.score)

10


## __slot__函数
用于限制可以对某个类添加的属性

In [12]:
#限制允许添加的属性，用tuple定义
class S(object):
    __slots__=('name','age')

s2=S()
s2.name='Jay'

In [13]:
s2.score=9

AttributeError: 'S' object has no attribute 'score'

# 使用@property

In [14]:
class Student(object):
    def get_score(self):
        return self._score

    def set_score(self, value):
        if not isinstance(value, int):
            #使用错误抛出实现报错提醒
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value


In [15]:
s=Student()
s.set_score(10)

In [16]:
s.set_score(1000)

ValueError: score must between 0 ~ 100!

## @property
将类的方法变成属性调用，使得更加直观方便，一般是set类方法

In [17]:
#原方法调用
s.set_score(20)
print(s.get_score())

20


In [18]:
#通过property实现方法转属性
class Student(object):
    
    #这个是getter方法，直接用property修饰即可
    @property
    def score(self):
        return self._score

    #这个是由property衍生的setter方法
    @score.setter
    def score(self,value):
        if not isinstance(value,int):
            raise ValueError('Score must be an integer!')
        if value<0 or value>100:
            raise ValueError('Score must between 0~100!')
        
        self._score=value


In [19]:
s4=Student()
s4.score=60 #相当于s.score(60)

s4.score    #相当于s.score()

60

In [20]:
#定义只读元素,对于没有setter方法的便是只读元素
class Student(object):
    
    @property
    def birth(self):
        return self._birth  #注意这里的属性名不能跟方法名重，会导致错误
    
    @birth.setter
    def birth(self,value):
        self._birth=value
    
    #对于age即是一个只读元素
    @property
    def age(self):
        return 2025-self._birth

# 多重继承
很多情况下，可能根据一个类别的多种属性，需要进行多重分类继承

In [21]:
#第一大类   动物
class Animal(object):
    pass

#第二大类   类别动物
class Mammal(Animal):
    pass
class Bird(Animal):
    pass

#第三大类   具体动物
class Dog(Mammal):
    pass
class Bat(Mammal):
    pass
class Parrot(Bird):
    pass
class Ostrich(Bird):
    pass

In [22]:
#如果要继承特定功能
class Runnable(object):
    def run(self):
        print('Running')

class Flyable(object):
    def fly(self):
        print('Flying')

#进行多继承
class Dog(Mammal,Runnable):
    pass

## MixIn
一般为了更好体现继承的类别，对于增加功能的继承，命名附加MixIn

In [23]:
#可跑、肉食
class RunnableMixIn(object):
    pass

class CarnivorousMixIn(object):
    pass

class Dog(Mammal,RunnableMixIn,CarnivorousMixIn):
    pass

# 定制类

## __ str __
返回一个较为好看的字符串

In [ ]:
class Student(object):
    def __init__(self,name):
        self.name=name

print(Student('Michael'))   #并不雅致

In [ ]:
#定义一个更雅致的
class Student(object):
    def __init__(self,name):
        self.name=name
    def __str__(self):
        return 'Student object (name: %s)' %self.name
    
print(Student('Michael'))   #调用__str__时

Student object (name: Michael)


In [ ]:
s=Student('Michael')
s   #直接显示变量调用的是__repr__()

## __ iter __
可迭代的类实现必须

In [ ]:
class Fib(object):
    #初始化
    def __init__(self):
        self.a,self.b=0,1

    #迭代对象——为实例本身
    def __iter__(self):
        return self
    
    #下一个迭代
    def __next__(self):
        self.a,self.b=self.b,self.a+self.b
        if self.a>10000:
            raise StopIteration()
        return self.a

#可迭代类即可用于循环:不断调用内部__next__实现迭代
for n in Fib():
    print(n)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765


## __ getitem __
类似list一样取出元素的方法

In [33]:
class Fib(object):
    def __getitem__(self,n):
        a,b=1,1
        for x in range(n):
            a,b=b,b+a
        return a

#实现访问
f=Fib()
print(f[0])
print(f[3])

1
3


## __ getattr __
对于不想设置的属性，又想调用，可以用该方法实现

In [35]:
class Student(object):
    def __init__(self):
        self.name='Michael'
    
    def __getattr__(self,attr):
        if attr=='score':
            return 99
        
s=Student()
s.score

99

## __ call __
实现直接对实例调用

In [42]:
class Student(object):
    def __init__(self,name):
        self.name = name
    
    def __call__(self):
        print('My name is %s' % self.name)
    
s=Student('Michael')
s()

My name is Michael


# 使用枚举类
定义常量时，常用int，但是这样本质依旧是变量。
更好的办法是为这样的枚举类型定义一个class，每个常量为其唯一的实例

In [44]:
from enum import Enum

Month=Enum('Month',('Jan','Feb','Mar')) #获得Month枚举类

for name,member in Month.__members__.items():
    print(name,'=>',member,',',member.value)

Jan => Month.Jan , 1
Feb => Month.Feb , 2
Mar => Month.Mar , 3
